In [1]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz

C:\ProgramData\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
# products_sheet_id = "1FB2bbvWE2nsf49KO_DkWzbL2hm9r4G8j"
# products_list = "Tablib%20Dataset"
# url = f"https://docs.google.com/spreadsheets/d/{products_sheet_id}/gviz/tq?tqx=out:csv&sheet={products_list}"

In [2]:
# convert csv datasets into dataframes

pdt_df = pd.read_csv('full_products_list.csv')
batches_df = pd.read_csv('batches_list.csv')
recipe_df = pd.read_csv('recipe_list.csv')

In [3]:
# import excel file from an absolute path with pandas

xls = pd.ExcelFile(r'C:\Users\NDA\Desktop\EVE\TMCG\documents\duplicate_products.xlsx')

In [4]:
# convert xls datasets on sheet names into dataframes

dp_list = pd.read_excel(xls, 'copy_final_list_duplicates')
unique_pdts_df = pd.read_excel(xls, 'unique final list')

In [5]:
# drop unwanted columns from the duplicates list dataframe

new_dp_list = dp_list.drop(['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 8',
                           'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14',
                           'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20',
                           'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23'], axis=1)

In [6]:
# cleanup data by renaming pandas series with more meaningfull names

new_dp_list = new_dp_list.rename(columns={'Eva': 'PRODUCT ID', 'Unnamed: 6': 'Generic Name', 'Unnamed: 7': 'Brand Name'})

In [7]:
# drop null values based on rows

new_dp_list = new_dp_list.dropna(axis='rows')

In [ ]:
# check to see if dataset is now clean 

new_dp_list.tail()

In [8]:
# remove the inverse labels

new_dp_list = new_dp_list.drop([new_dp_list.index[-1]])

In [ ]:
# check to see whether id has numerical values
# re-check after running next cell

new_dp_list.info()

In [9]:
# convert product id into numeric 

new_dp_list['PRODUCT ID'] = pd.to_numeric(new_dp_list['PRODUCT ID'], errors='ignore')

In [10]:
new_dp_list.head()

,PRODUCT ID,Generic Name,Brand Name
0,3951,Betadine Mouthwash Gargle,Betadine Mouthwash Gargle
1,387,Montelukast,DENK AIR JUNIOR 5mg
2,4933,Montelukast Tabs,Denk-air
3,4942,Montelukast Tabs,Denk-Air Junior
4,3146,Diclofenac Tabs,Diclomol SR


In [11]:
# check unique products dataframe

unique_pdts_df.columns

Index(['ID', 'COUNTRY OF ORIGIN', 'FORMULATION',
       'RETAIL PRICE PER SMALLEST UNIT', 'RETAIL PRICE PER SMALLEST UNIT.1',
       'PACK SIZE', 'GENERIC NAME', 'BRAND NAME', 'STRENGTH', 'eshop_id',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13',
       'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17',
       'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21',
       'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25'],
      dtype='object')

In [12]:
# drop unnecessary columns from unique lists

unique_pdts_df = unique_pdts_df.drop(['COUNTRY OF ORIGIN', 'FORMULATION', 'RETAIL PRICE PER SMALLEST UNIT', 
                                      'RETAIL PRICE PER SMALLEST UNIT.1', 'PACK SIZE', 'STRENGTH', 'eshop_id', 
                                      'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 
                                      'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 
                                      'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 
                                      'Unnamed: 25'], axis=1)

In [13]:
unique_pdts_df.head()

,ID,GENERIC NAME,BRAND NAME
0,5302,Glyceryl trinitrate,accord 500mcg
1,3949,Betadine Antiseptic Solution,Betadine Antiseptic Solution
2,715,Montelukast,DENK AIR 10MG
3,4328,Diclofenac SR Tabs,Diclomol
4,830,Doxazosin,DOXAZOSIN 4MG UK


In [14]:
# change column name into product id for easy manipulation

unique_pdts_df = unique_pdts_df.rename(columns={'ID': 'PRODUCT ID'})

In [15]:
# give proper indices to the dataset

new_dp_list.reset_index(drop=True, inplace=True)

In [16]:
# create a new column on which we will compare our product ids
new_dp_list['NEW ID'] = ''

# create a new column on which we will compare our generic names
new_dp_list['new_generic_name'] = ''

# loop through the unique products dataframe using its index
for i in range(len(unique_pdts_df)):
    
   # loop through the list of duplicates using its index
    for j in range(len(new_dp_list)):
        
       # compare the two pandas series on generic names. Coerce them to lowercase for proper comparison
        if unique_pdts_df['GENERIC NAME'][i].lower() in new_dp_list['Generic Name'][j].lower():
            
            # assign the new ids to the resulting product from unique products loop and do the same for generic name
            new_dp_list['NEW ID'][j] = unique_pdts_df['PRODUCT ID'][i]
            new_dp_list['new_generic_name'][j] = unique_pdts_df['GENERIC NAME'][i]        

<ipython-input-16-3f3e64898cba>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_dp_list['NEW ID'][j] = unique_pdts_df['PRODUCT ID'][i]
<ipython-input-16-3f3e64898cba>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_dp_list['new_generic_name'][j] = unique_pdts_df['GENERIC NAME'][i]


In [17]:
# subset the rows that were filled
filled = new_dp_list[new_dp_list['NEW ID'] != '']

# filled

In [18]:
# subset the unfilled rows
unfilled = new_dp_list[new_dp_list['NEW ID'] == '']

# unfilled

In [19]:
# re-index the dataframe because dropping out rows has messed up with its indexing

unfilled.reset_index(drop=True, inplace=True)

In [20]:
# loop through the unique products dataframe using its index
for i in range(len(unique_pdts_df)):
    
    # loop through the list of duplicates using its index
    for j in range(len(unfilled)):
        
        # extract string from the product
        str_a  = unique_pdts_df['GENERIC NAME'][i].lower()
        str_b = unfilled['Generic Name'][j].lower()
        
        # assign ratio to both strings
        ratio = fuzz.partial_ratio(str_b, str_a)            
        
        # sent conditon for comparing the series
        if ratio >= 70:
            unfilled['new_generic_name'][j] = unique_pdts_df['GENERIC NAME'][i]
            unfilled['NEW ID'][j] = unique_pdts_df['PRODUCT ID'][i]

<ipython-input-20-97984ad8ed7d>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unfilled['new_generic_name'][j] = unique_pdts_df['GENERIC NAME'][i]
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
<ipython-input-20-97984ad8ed7d>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unfilled['NEW ID'][j] = unique_

In [21]:
# unfilled

In [22]:
# combine the dataframes into one dataframe
combined_df = pd.concat([unfilled, filled], axis=0).sort_values('Generic Name')

In [23]:
# re-index the dataframe
combined_df.reset_index(drop=True, inplace=True)

In [24]:
# combined_df

### The product dataset

In [25]:
# investigate the products on columns
pdt_df.columns

Index(['ID', 'COUNTRY OF ORIGIN', 'FORMULATION',
       'RETAIL PRICE PER SMALLEST UNIT', 'RETAIL PRICE PER SMALLEST UNIT.1',
       'PACK SIZE', 'GENERIC NAME', 'BRAND NAME', 'STRENGTH', 'eshop_id'],
      dtype='object')

In [ ]:
# new_pdt_df= pdt_df.drop(['COUNTRY OF ORIGIN', 'FORMULATION', 'PACK SIZE', 'RETAIL PRICE PER SMALLEST UNIT', 
#              'RETAIL PRICE PER SMALLEST UNIT.1', 'PACK SIZE', 'STRENGTH', 'eshop_id'], axis=1)

In [26]:
# investigate the product's types
pdt_df.dtypes

ID                                    int64
COUNTRY OF ORIGIN                    object
FORMULATION                          object
RETAIL PRICE PER SMALLEST UNIT      float64
RETAIL PRICE PER SMALLEST UNIT.1    float64
PACK SIZE                             int64
GENERIC NAME                         object
BRAND NAME                           object
STRENGTH                             object
eshop_id                            float64
dtype: object

In [27]:
new_dp_list.head()

,PRODUCT ID,Generic Name,Brand Name,NEW ID,new_generic_name
0,3951,Betadine Mouthwash Gargle,Betadine Mouthwash Gargle,3352,Betadine Mouthwash Gargle
1,387,Montelukast,DENK AIR JUNIOR 5mg,715,Montelukast
2,4933,Montelukast Tabs,Denk-air,715,Montelukast
3,4942,Montelukast Tabs,Denk-Air Junior,715,Montelukast
4,3146,Diclofenac Tabs,Diclomol SR,,


In [28]:
# change product's id for ease in comparing with other dataframes
new_pdt_df = pdt_df.rename(columns={'ID': 'PRODUCT ID'})


In [29]:
# check to find out whether the new_product_df id is identical to the duplicate list id 
merged_pdt_df = new_pdt_df['PRODUCT ID'].isin(new_dp_list['PRODUCT ID'])

In [30]:
# drop the identical ids
new_pdt_df.drop(new_pdt_df[merged_pdt_df].index, inplace=True)

In [31]:
# checkout the new dataframe
new_pdt_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2554 entries, 0 to 2603
Data columns (total 10 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   PRODUCT ID                        2554 non-null   int64  
 1   COUNTRY OF ORIGIN                 1218 non-null   object 
 2   FORMULATION                       1484 non-null   object 
 3   RETAIL PRICE PER SMALLEST UNIT    2532 non-null   float64
 4   RETAIL PRICE PER SMALLEST UNIT.1  2483 non-null   float64
 5   PACK SIZE                         2554 non-null   int64  
 6   GENERIC NAME                      2554 non-null   object 
 7   BRAND NAME                        2554 non-null   object 
 8   STRENGTH                          2229 non-null   object 
 9   eshop_id                          389 non-null    float64
dtypes: float64(3), int64(2), object(5)
memory usage: 219.5+ KB


In [33]:
# test on a product from the duplicates list
new_pdt_df[new_pdt_df['PRODUCT ID'] == 3951]

,PRODUCT ID,COUNTRY OF ORIGIN,FORMULATION,RETAIL PRICE PER SMALLEST UNIT,RETAIL PRICE PER SMALLEST UNIT.1,PACK SIZE,GENERIC NAME,BRAND NAME,STRENGTH,eshop_id


In [ ]:
# merged_pdt_df[merged_pdt_df['PRODUCT ID'] == 3951]

In [ ]:
### The batches dataframe

In [ ]:
merged_pdt_df.info()

In [ ]:
new_batches_df= batches_df.drop(['id', 'created', 'modified', 'initial_item_count', 
             'cost_price', 'item_count', 'expiration_date', 'manufacturing_date'], axis=1)

In [ ]:
# new_batches_df.info()

In [ ]:
# recipe_df.columns

In [ ]:
new_recipe_df = recipe_df.drop(['id', 'created', 'modified', 'prescription', 'dosage', 'tablets', 'quantity', 'frequency', 
                                'duration', 'route', 'price_discount', 'serving_pharmacy', 'price_controller', 'rx_price',
                               'external_price', 'requires_approval', 'approved', 'dropped', 'canceling_party', 
                                'reason_for_cancelation', 'authorized_by_tmcg', 'authorized_by_insurer', 'client_decision', 
                                'insurer_code'], axis=1)

In [ ]:
# new_recipe_df.info()

In [ ]:
# new_recipe_df.head()

In [ ]:
dp_list.head()

In [ ]:
# unique_pdts_df.info()

In [ ]:
# dp_list.columns

In [ ]:
# dp_list.info()

In [ ]:
new_dp_list.head()

In [ ]:
new_dp_list.info()

In [ ]:
# unique_pdts_df.head(49)

In [ ]:
# new_dp_list['PRODUCT ID'].to_list()

In [ ]:
new_dp_list.tail()

In [ ]:
new_dp_list.tail()

In [ ]:
new_dp_list.info()

In [ ]:
# new_dp_list['PRODUCT ID'].to_list()

In [ ]:
new_pdt_df = new_pdt_df.rename(columns={'ID': 'PRODUCT ID'})

In [ ]:
new_pdt_df.head()

In [ ]:
merged_pdt_dup = pd.merge(new_dp_list, new_pdt_df, on=['PRODUCT ID'], how='inner')

In [ ]:
# merged_pdt_dup.head(51)

In [ ]:
merged_batch_dup = pd.merge(new_dp_list, new_batches_df, on=['PRODUCT ID'], how='inner')

In [ ]:
merged_batch_dup.info()

In [ ]:
new_dp_list.shape

In [ ]:
# new_recipe_df

In [ ]:
new_recipe_df = new_recipe_df.rename(columns={'product_id': 'PRODUCT ID'})

In [ ]:
new_recipe_df.head()

In [ ]:
merged_recipe_dup = pd.merge(new_dp_list, new_recipe_df, on=['PRODUCT ID'], how='inner')

In [ ]:
merged_recipe_dup.info()

In [ ]:
# new_pdt_df
# new_batches_df
# new_recipe_df

# merged_pdt_dup
# merged_batch_dup
# merged_recipe_dup

In [ ]:
unique_pdts_df.head()

In [ ]:
merged_batch_dup.head()

In [ ]:
new_batch = merged_batch_dup.replace()

In [ ]:
# for prod in merged_batch_dup['Generic Name']:
#     if np.where(merged_batch_dup['Generic Name'] == unique_pdts_df['GENERIC NAME'], True, False):
#         print(prod)

In [ ]:
merged_series = merged_batch_dup['Generic Name']
unique_series = unique_pdts_df['GENERIC NAME']

In [ ]:
merged_batch_dup.columns

In [ ]:
# merged_row = merged_batch_dup.loc[['Montelukast']]
# merged_row

In [ ]:
merged_batch_dup['Generic Name'].unique()

In [ ]:
unique_pdts_df['GENERIC NAME'].unique()

In [ ]:
unique_pdts_df.columns

In [ ]:
pdt_df.head()

In [ ]:
pdt_df['NEW ID'] = ""

In [ ]:
for i in range(len(unique_pdts_df)):
    for j in range(len(pdt_df)):
#         if unique_pdts_df['GENERIC NAME'][i] == pdt_df['GENERIC NAME'][j]:
#             pdt_df['NEW ID'][j] = unique_pdts_df['PRODUCT ID'][i]
        if unique_pdts_df['GENERIC NAME'][i].lower() in pdt_df['GENERIC NAME'][j].lower():
            pdt_df['NEW ID'][j] = unique_pdts_df['PRODUCT ID'][i]

In [ ]:
# unique_pdts_df[unique_pdts_df['GENERIC NAME'] == 'Montelukast']

In [ ]:
pdt_df.head()

In [ ]:
unique_pdts_df.head()

In [ ]:
pdt_df[pdt_df['NEW ID'] != ""]

In [ ]:
list(pdt_df['GENERIC NAME'].unique())

In [ ]:
unique_pdts_df['GENERIC NAME'].unique()

In [ ]:
'Montelukast'in(pdt_df['GENERIC NAME'])

In [ ]:
new_dp_list.head(20)

In [ ]:
new_dp_list.columns

In [ ]:
new_dp_list[new_dp_list['Generic Name'] == 'Diclofenac 12.5mg Suppositories']

In [ ]:
unique_pdts_df[unique_pdts_df['GENERIC NAME'] == 'Diclofenac']

In [ ]:
new_dp_list.head(10)

In [ ]:
print(new_dp_list['Generic Name'])

In [ ]:
new_dp_list.info()

In [ ]:
unique_pdts_df[unique_pdts_df['GENERIC NAME'].str.contains('Loratadine')]

In [ ]:
pattern = "+./'"

In [ ]:
unique_pdts_df.loc['Diclofenac 12.5mg Suppositories', 'GENERIC NAME']

In [ ]:
#         if unique_pdts_df['GENERIC NAME'][i] == pdt_df['GENERIC NAME'][j]:
#             pdt_df['NEW ID'][j] = unique_pdts_df['PRODUCT ID'][i]
#         elif new_dp_list['Generic Name'][j].lower().contains(pat="+./'", regex=True) in new_dp_list['Generic Name'][j].lower():
#             new_dp_list['NEW ID'][j] = unique_pdts_df['PRODUCT ID'][i]